In [1]:
import keras
import tensorflow as tf

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

#Importing the VAE and RNN.
import os
import sys
#Adding WorldModels path to pythonpath
nb_dir = os.path.split(os.getcwd())[0]
print(nb_dir)
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

Using TensorFlow backend.


/home/kaiolae/code/word_models_keras_test/WorldModels


In [ ]:

import analyze_rnn_predictions
%load_ext autoreload
%autoreload 2 #If we want to automatically load external modules as they are updated.

In [ ]:
#To access gym
import sys
sys.path.append('/home/kaiolae/.virtualenvs/worldmodels/lib/python3.5/site-packages')
import os

In [ ]:
import fnmatch
def get_all_models():
    all_models = {} #Dict of all models, with num mixtures as key
    for folder in os.listdir('../'):
        
        if fnmatch.fnmatch(folder, 'trained_sequential_rnn_*') :
            mixtures_start_index = folder.find('rnn_')+4
            mixtures_end_index = folder.find('mixtures')
            num_mixtures = int(folder[mixtures_start_index:mixtures_end_index])
            if num_mixtures in all_models.keys():
                all_models[num_mixtures].append("../"+folder)
            else:
                all_models[num_mixtures] = ["../"+folder]
            
    return all_models
            
print(get_all_models())

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML


def plot_movie_mp4(image_array):
    dpi = 2.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    #fig = plt.figure(figsize=(1,1), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=len(image_array))
    display(HTML(anim.to_html5_video()))

In [ ]:
from collections import namedtuple

rnn_analyzers = {}
for key, val in get_all_models().items():
    rnn_analyzers[key] = []
    for model in val:
        if "run10" in model or "run5" in model or "run15" in model:
            continue #TODO remove
        print("Loading model: ", model)
        new_analyzer = analyze_rnn_predictions.RNNAnalyzer(model+"/rnn_trained_model.h5", key, 0.2) #TODO Note, I previously used temp=0.1 - Which Ha noted was so low that we got mode collapse.
        rnn_analyzers[key].append(new_analyzer)
print("rnn analyzers: ", rnn_analyzers)

In [ ]:
#Get a start video
#Getting data to feed into the VAE and RNN
import numpy as np
import os
# load preprocessed data
training_data = np.load(os.path.join("../rnn_data_64_dim", "rnn_training_data.npz"))
action_data = training_data['action']
observation_data = training_data['latent']

episode_num = 16
single_action_sequence = action_data[episode_num]
single_latent_sequence = observation_data[episode_num]
print("Actions length: ", len(single_action_sequence))
print("Latent vectors length: ", len(single_latent_sequence))
num_frames = len(single_latent_sequence)



#Decode all 100 vectors, store as video
#Visualize video.

In [ ]:
#Decode a sequence with the VAE and visualize it
def decode_and_visualize(latent_vector_sequence, rnn_analyzer):
    plot_movie_mp4(rnn_analyzer.decode_with_vae(latent_vector_sequence))

### Visualizing one sequence of latent vectors from the training data

In [ ]:
#From the actual sampled images. All loss here is due to VAE.
rnn1_analyzer = rnn_analyzers[2][0]
decode_and_visualize(single_latent_sequence, rnn1_analyzer)

In [ ]:
#Generating a dream.
action = 0 #Consider random action. Or, when predicting step by step, use the actual action
dreamed_zs1 = []
dreamed_z, mixture_weights = rnn1_analyzer.predict_one_step(action, previous_z=single_latent_sequence[0])
dreamed_zs1.append(dreamed_z)
print(dreamed_z)
for i in range(100):
    dreamed_z, mixture_weights = rnn1_analyzer.predict_one_step(action)
    dreamed_zs1.append(dreamed_z)
    

In [ ]:
#Visualizing the dream
decode_and_visualize(dreamed_zs1, rnn1_analyzer)

In [ ]:
#Generating a dream with 16 mixtures
action = 0 #Consider random action. Or, when predicting step by step, use the actual action
dreamed_zs16 = []
dreamed_z = rnn16_analyzer.predict_one_step(action, previous_z=single_latent_sequence[0])
dreamed_zs16.append(dreamed_z)
for i in range(100):
    dreamed_zs16.append(rnn16_analyzer.predict_one_step(action))
#Visualizing the dream
decode_and_visualize(dreamed_zs16, rnn16_analyzer)

# 

In [ ]:
vae = rnn_analyzers[2][0].vae
print(single_latent_sequence.shape)
vae.decoder.predict(single_latent_sequence)

In [ ]:
#TODO Make sure to "warm up" the LSTM with many inputs before recording."

#TODO Without reset, I got the expected result. But with reset, differences between RNNs disappear.
#Do differences for some reason only appear after a (large) number of iterations??
#One solution might be to run 1000 timesteps, to ensure we get into the internal state that
#exposes differences.
#Also, randomizing initial_z could help.
#Yes, longer sequences seemed to work.

#TODO A better measure of creativity is differences between consecutive frames. Did a ball appear/disappear, etc?
#Test plotting that with "fireball_delta"/monster_delta

import pandas as pd
from test_creativity_stability import count_events_on_trained_rnn, count_appearances_and_disappearances
#Comparing performance of different models
initial_z = single_latent_sequence[0]
EPISODE_LENGTHS = 100
zero_actions = [0 for i in range(EPISODE_LENGTHS)]

#TODO These should both instead reflect the difference from initial_z, since we want to
#measure creativity.

#Collecting data for all treatments
dream_data = []
NUM_STATISTICAL_TESTS = 1

for i in range(NUM_STATISTICAL_TESTS):
    print(i)
    for num_mixtures, rnn_model_list in rnn_analyzers.items():
        
        for rnn_model in rnn_model_list:
            print("analyzing ", num_mixtures)            
            rnn_model._reset()
            rnn_model.warm_up_lstm(single_action_sequence, single_latent_sequence)
            e = count_events_on_trained_rnn(rnn_model.vae, rnn_model, initial_z, zero_actions, EPISODE_LENGTHS)
            d = count_appearances_and_disappearances(rnn_model.vae, rnn_model, initial_z, zero_actions, EPISODE_LENGTHS)
            combined_dict = {**e, **d}
            combined_dict["num_mixtures"] = num_mixtures
            dream_data.append(combined_dict)
            
#Turning dream-data entries into dataframes, one for each treatment
dream_dataframes = pd.DataFrame(dream_data)
print(dream_dataframes)

#TODO Something is wrong in my measurement of fireballs delta. It shouldn't be possible that it's bigger than
#num_fireballs.
#TODO should be easy to dump to file also now.
#At some point, I should split into a generate-and-dump script (this), and a load-and-analyze script.
#dream_

#TODO Not easy to see any clear effects of modifying number of mixtures.

In [ ]:
for num_mixtures in fireballs_dreamed.keys():
    fireball_dreams = fireballs_dreamed[num_mixtures]
    print("avg fireballs for ", num_mixtures, ": ", sum(fireball_dreams)/float(len(fireball_dreams)))
    monster_dreams = monsters_dreamed[num_mixtures]

In [ ]:
print(fb1)

In [ ]:
#Testing differences between real and predicted images. Rather than dreaming up many steps ahead, we here
#Dream 1 step ahead, and compare to what actually happened. Tests stability of the prediction.
errors = [] #With 1 entry for each model, showing its parameters and results
initial_z = single_latent_sequence[0]

NUM_STATISTICAL_TESTS = 1
from test_creativity_stability import count_differences_between_reality_and_prediction

print(single_latent_sequence.shape)
#TODO Is this short sequence hiding differences? Making a fake long sequence by concatenating.
extended_latent_sequence = []
for i in range(10):
    for latent in single_latent_sequence:
        extended_latent_sequence.append(latent)
extended_latent_sequence = np.array(extended_latent_sequence)
#padded_action_sequence = np.append(single_action_sequence, 0)
extended_actions = np.repeat(single_action_sequence, 10)

print(extended_actions.shape)
print(extended_latent_sequence.shape)

for i in range(NUM_STATISTICAL_TESTS):
    for num_mixtures, rnn_model_list in rnn_analyzers.items():
        for rnn_model in rnn_model_list:      
            print("Analyzing ", rnn_model)
            rnn_model._reset(initial_z)
            e = count_differences_between_reality_and_prediction(rnn_model.vae, rnn_model, extended_latent_sequence, extended_actions)
            e['num_mixtures'] = num_mixtures
            errors.append(e)
            
#Turning entries into dataframes, one for each treatment
error_dataframes = pd.DataFrame(errors)
print(error_dataframes)


In [ ]:
for num_mixtures in fireballs_dreamed.keys():
    err = errors[num_mixtures]
    print("avg errors for ", num_mixtures, ": ", sum(err)/float(len(err)))

#Preliminary conclusion: Many mixtures (8/16) give more creativity, being able to "imagine" more fireballs,
#but also more inaccuracies, by imagining fireballs that shouldnt be there.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
#print(sns.load)
g=sns.boxplot(x="num_mixtures", y="num_fireballs", data=dream_dataframes)
g.set(ylim=(None, 1000))
sns.despine()
#plt.show()

#Plotting fireballs per episode

In [ ]:
sns.boxenplot(x="num_mixtures", y="fireball_delta", data=dream_dataframes)
sns.despine()

In [ ]:
sns.boxenplot(x="num_mixtures", y="imagined_fireballs", data=error_dataframes)
sns.despine()

In [ ]:
sns.boxenplot(x="num_mixtures", y="imagined_monsters", data=error_dataframes)
sns.despine()

In [ ]:
sns.boxenplot(x="num_mixtures", y="missing_monsters", data=error_dataframes)
sns.despine() #TODO Seems to be no disadvantage to being creative here - 16 gives low errors. Maybe if I let it dream for more frames?

In [ ]:
sns.boxenplot(x="num_mixtures", y="missing_fireballs", data=error_dataframes)
sns.despine()

In [ ]:
#Plotting monsters per episode
#print(sns.load)
monsters_data=[]
for error_dataframes
monsters_data = np.array([m1, m4, m8, m16])
monsters_data = pd.DataFrame(monsters_data.T)
monsters_data.columns = ['1', '4', '8', '16']

sns.boxenplot(data=monsters_data, palette='Blues')
sns.despine()
#plt.show()

boxplot_data.describe()

In [ ]:
#Statistical test
from scipy.stats import mannwhitneyu
print("statistical difference in fireballs with 1 vs 4 mixtures")
print(mannwhitneyu(fb1, fb4))
print("statistical difference in fireballs with 4 vs 16 mixtures")
print(mannwhitneyu(fb4, fb16))

In [ ]:
#Missing fireballs
missing_fireballs_data = np.array([mf1, mf4, mf8, mf16])
missing_fireballs_data = pd.DataFrame(missing_fireballs_data.T)
missing_fireballs_data.columns = ['1', '4', '8', '16']
sns.boxenplot(data=missing_fireballs_data)


In [ ]:
#Imagined fireballs
imagined_fireballs_data = np.array([if1, if4, if8, if16])
imagined_fireballs_data = pd.DataFrame(imagined_fireballs_data.T)
imagined_fireballs_data.columns = ['1', '4', '8', '16']
sns.boxenplot(data=imagined_fireballs_data)

In [ ]:
#Total prediction errors
prediction_errors = np.array([errors1, errors4, errors8, errors16])
prediction_errors_data = pd.DataFrame(prediction_errors.T)
prediction_errors_data.columns = ['1', '4', '8', '16']
sns.boxenplot(data=prediction_errors_data)

In [ ]:
#Making a "pareto plot" of creativity vs precision
#TODO Change my measurements, to measure these two on the same rollouts.
#TODO Each datapoint should represent a single model - I need to run multiple trainings of each model.
creativity_precision_data=[] #Tuples (treatment, creativity, precision)
all_precision_data = [errors1, errors4, errors8, errors16] #TODO Replace with more complete data
all_creativity_data = [fb1, fb4, fb8, fb16]
creativity_prediction_data_summed=[]
for treatment_counter in range(len(all_precision_data)):
    treatment_precision = all_precision_data[treatment_counter]
    treatment_creativity = all_creativity_data[treatment_counter]
    current_treatment_summed=[0, 0]
    for i in range(len(treatment_precision)):
        creativity_precision_data.append(["Treatment"+str(treatment_counter), treatment_precision[i], treatment_creativity[i]])
        current_treatment_summed[0]=current_treatment_summed[0]+treatment_precision[i]
        current_treatment_summed[1]=current_treatment_summed[1]+treatment_creativity[i]
    creativity_prediction_data_summed.append(current_treatment_summed)
print(creativity_precision_data)

In [ ]:
#TODO May become quite interesting with more data points.
creativity_precision_data_summed = np.array(creativity_precision_data_summed)
print(creativity_precision_data_summed)
creativity_precision_dataframe = pd.DataFrame({"Treatment":["1","4","8","16"],
                                               "precision-error":creativity_precision_data_summed[:,0],
                                               "creativity":creativity_precision_data_summed[:,1]})
sns.scatterplot(x="precision-error", y="creativity", data=creativity_precision_dataframe, style="Treatment")

In [ ]:

creativity_precision_data=np.array(creativity_precision_data)
creativity_precision_dataframe = pd.DataFrame({"Treatment":creativity_precision_data[:,0],
                                               "precision-error":creativity_precision_data[:,1],
                                               "creativity":creativity_precision_data[:,2]})
creativity_precision_dataframe = creativity_precision_dataframe.apply(pd.to_numeric, errors='ignore')
print(creativity_precision_dataframe.dtypes)
print(creativity_precision_dataframe)
sns.scatterplot(x="creativity", y="precision-error", data=creativity_precision_dataframe, style="Treatment")

In [ ]:
tips = sns.load_dataset("tips")
print(tips)
sns.scatterplot(x="tip", y="total_bill", data=tips)